In [1]:
# Dependencies

import requests
from config import api_key
import pandas as pd
import datetime as dt
from datetime import datetime as ds
import json
from pprint import pprint
import sqlalchemy
from sqlalchemy import create_engine

### Getting Inflation data for US by reading HTML

In [2]:
# Define the url to read the html

url = 'https://www.in2013dollars.com/us/inflation/1930?amount=1'

In [3]:
# Read the web page and print results

cpi_inflation = pd.read_html(url)
cpi_inflation

[                            0               1
 0     Cumulative price change       1,486.09%
 1      Average inflation rate           3.08%
 2  Converted amount ($1 base)          $15.86
 3  Price difference ($1 base)          $14.86
 4                 CPI in 1930          16.700
 5                 CPI in 2021         264.877
 6           Inflation in 1930          -2.34%
 7           Inflation in 2021           2.62%
 8                  $1 in 1930  $15.86 in 2021,
     Year Dollar Value Inflation Rate
 0   1930        $1.00         -2.34%
 1   1931        $0.91         -8.98%
 2   1932        $0.82         -9.87%
 3   1933        $0.78         -5.11%
 4   1934        $0.80          3.08%
 ..   ...          ...            ...
 87  2017       $14.68          2.13%
 88  2018       $15.04          2.49%
 89  2019       $15.31          1.76%
 90  2020       $15.50          1.23%
 91  2021       $15.86         2.34%*
 
 [92 rows x 3 columns],
                  Initial value              Eq

In [4]:
# Select the required table for CPI and Inflation from the page and build the required dataframe

cpi_inflation_df = cpi_inflation[1]
cpi_inflation_df

,Year,Dollar Value,Inflation Rate
0,1930,$1.00,-2.34%
1,1931,$0.91,-8.98%
2,1932,$0.82,-9.87%
3,1933,$0.78,-5.11%
4,1934,$0.80,3.08%
...,...,...,...
87,2017,$14.68,2.13%
88,2018,$15.04,2.49%
89,2019,$15.31,1.76%
90,2020,$15.50,1.23%


In [5]:
# Rename Columns

cpi_inflation_df.columns = ['date', 'dollar_value', 'inflation_rate']
cpi_inflation_df

,date,dollar_value,inflation_rate
0,1930,$1.00,-2.34%
1,1931,$0.91,-8.98%
2,1932,$0.82,-9.87%
3,1933,$0.78,-5.11%
4,1934,$0.80,3.08%
...,...,...,...
87,2017,$14.68,2.13%
88,2018,$15.04,2.49%
89,2019,$15.31,1.76%
90,2020,$15.50,1.23%


### Getting GDP data from St. Louis Fed. using an API call

In [6]:
#https://alfred.stlouisfed.org/series/downloaddata?seid=MABMM301USA657S

url = 'https://api.stlouisfed.org/fred/series/observations?series_id=A191RL1A225NBEA'
realtime_start = '1958-01-01'
realtime_end = '2021-01-31'

In [7]:
# Build query URL

query_url = url + "&realtime_start=" + realtime_start + "&realtime_end=" + realtime_end + "&api_key=" + 
api_key + "&file_type=json"

In [8]:
# Print the results of the query in a json format

response = requests.get(query_url).json()
pprint(response)

{'count': 129,
 'file_type': 'json',
 'limit': 100000,
 'observation_end': '9999-12-31',
 'observation_start': '1600-01-01',
 'observations': [{'date': '1930-01-01',
                   'realtime_end': '2021-01-31',
                   'realtime_start': '2014-10-31',
                   'value': '-8.5'},
                  {'date': '1931-01-01',
                   'realtime_end': '2021-01-31',
                   'realtime_start': '2014-10-31',
                   'value': '-6.4'},
                  {'date': '1932-01-01',
                   'realtime_end': '2021-01-31',
                   'realtime_start': '2014-10-31',
                   'value': '-12.9'},
                  {'date': '1933-01-01',
                   'realtime_end': '2018-07-26',
                   'realtime_start': '2014-10-31',
                   'value': '-1.3'},
                  {'date': '1933-01-01',
                   'realtime_end': '2021-01-31',
                   'realtime_start': '2018-07-27',
                   'v

In [9]:
# Select the observations from the response

pprint(response['observations'])

[{'date': '1930-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2014-10-31',
  'value': '-8.5'},
 {'date': '1931-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2014-10-31',
  'value': '-6.4'},
 {'date': '1932-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2014-10-31',
  'value': '-12.9'},
 {'date': '1933-01-01',
  'realtime_end': '2018-07-26',
  'realtime_start': '2014-10-31',
  'value': '-1.3'},
 {'date': '1933-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2018-07-27',
  'value': '-1.2'},
 {'date': '1934-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2014-10-31',
  'value': '10.8'},
 {'date': '1935-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2014-10-31',
  'value': '8.9'},
 {'date': '1936-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2014-10-31',
  'value': '12.9'},
 {'date': '1937-01-01',
  'realtime_end': '2021-01-31',
  'realtime_start': '2014-10-31',
  'value': '5.1'},
 {'date': '

In [10]:
# Dump the results into pandas dataframe

gdp_df = pd.DataFrame(response['observations'])
gdp_df

,realtime_start,realtime_end,date,value
0,2014-10-31,2021-01-31,1930-01-01,-8.5
1,2014-10-31,2021-01-31,1931-01-01,-6.4
2,2014-10-31,2021-01-31,1932-01-01,-12.9
3,2014-10-31,2018-07-26,1933-01-01,-1.3
4,2018-07-27,2021-01-31,1933-01-01,-1.2
...,...,...,...,...
124,2019-02-28,2020-07-29,2018-01-01,2.9
125,2020-07-30,2021-01-31,2018-01-01,3.0
126,2020-01-30,2020-07-29,2019-01-01,2.3
127,2020-07-30,2021-01-31,2019-01-01,2.2


In [11]:
# Select the required columns 'date' and 'value' for the analysis

gdp_df = gdp_df[['date', 'value']]
gdp_df

,date,value
0,1930-01-01,-8.5
1,1931-01-01,-6.4
2,1932-01-01,-12.9
3,1933-01-01,-1.3
4,1933-01-01,-1.2
...,...,...
124,2018-01-01,2.9
125,2018-01-01,3.0
126,2019-01-01,2.3
127,2019-01-01,2.2


In [12]:
# Rename Columns

gdp_df.columns = ['date', 'gdp_rate']
gdp_df

,date,gdp_rate
0,1930-01-01,-8.5
1,1931-01-01,-6.4
2,1932-01-01,-12.9
3,1933-01-01,-1.3
4,1933-01-01,-1.2
...,...,...
124,2018-01-01,2.9
125,2018-01-01,3.0
126,2019-01-01,2.3
127,2019-01-01,2.2


In [13]:
# Change the dtype of the Year column from 'object' to 'datetime' to be able to select the year portion of date

gdp_df['date'] = [ds.strptime(x,'%Y-%m-%d') for x in gdp_df['date']]

<ipython-input-13-39d9e4b90d6e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_df['date'] = [ds.strptime(x,'%Y-%m-%d') for x in gdp_df['date']]


In [14]:
# Select just the years from the dates

gdp_df['date'] = gdp_df['date'].dt.year
gdp_df

<ipython-input-14-c729f58d5b78>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_df['date'] = gdp_df['date'].dt.year


,date,gdp_rate
0,1930,-8.5
1,1931,-6.4
2,1932,-12.9
3,1933,-1.3
4,1933,-1.2
...,...,...
124,2018,2.9
125,2018,3.0
126,2019,2.3
127,2019,2.2


In [15]:
# Drop the duplicated year values

gdp_df = gdp_df.drop_duplicates(keep='last', subset=['date'])
gdp_df

,date,gdp_rate
0,1930,-8.5
1,1931,-6.4
2,1932,-12.9
4,1933,-1.2
5,1934,10.8
...,...,...
119,2016,1.7
123,2017,2.3
125,2018,3.0
127,2019,2.2


### Reading xls to obtain Money Stock (M3) in the US

In [16]:
# Read United States values M3 values from 1958 into a dataframe

m3_data = pd.read_excel('us_m3.xls')
m3_data

,observation_date,MABMM301USA657S_20210414
0,1958-01-01,NaN
1,1959-01-01,NaN
2,1960-01-01,3.744744
3,1961-01-01,6.753574
4,1962-01-01,7.778348
...,...,...
58,2016-01-01,6.787281
59,2017-01-01,5.700626
60,2018-01-01,3.848735
61,2019-01-01,5.096518


In [17]:
# Rename the columns

m3_data.columns = ['date', 'm3_rate']
m3_data

,date,m3_rate
0,1958-01-01,NaN
1,1959-01-01,NaN
2,1960-01-01,3.744744
3,1961-01-01,6.753574
4,1962-01-01,7.778348
...,...,...
58,2016-01-01,6.787281
59,2017-01-01,5.700626
60,2018-01-01,3.848735
61,2019-01-01,5.096518


In [18]:
#Change date format to year to match the CPI_Inflation table

m3_data['date'] = m3_data['date'].dt.year
m3_data = m3_data.round(decimals=2)
m3_data


,date,m3_rate
0,1958,NaN
1,1959,NaN
2,1960,3.74
3,1961,6.75
4,1962,7.78
...,...,...
58,2016,6.79
59,2017,5.70
60,2018,3.85
61,2019,5.10


### Create database connection and Load DataFrames into database

In [19]:
connection_string = "postgres:postgres@localhost:5432/economic_indicators"
engine = create_engine(f'postgresql://{connection_string}')

In [20]:
cpi_inflation_df.to_sql(name='cpi_inflation', con=engine, if_exists='append', index=False)

In [21]:
gdp_df.to_sql(name='gdp', con=engine, if_exists='append', index=False)

In [22]:
m3_data.to_sql(name='m3', con=engine, if_exists='append', index=False)

In [23]:
# Confirm tables exist in database
engine.table_names()

['cpi_inflation', 'gdp', 'm3']

### Query the tables

In [24]:
pd.read_sql_query('select * from cpi_inflation', con=engine).head()

,date,dollar_value,inflation_rate
0,1930,$1.00,-2.34%
1,1931,$0.91,-8.98%
2,1932,$0.82,-9.87%
3,1933,$0.78,-5.11%
4,1934,$0.80,3.08%


In [25]:
pd.read_sql_query('select * from gdp', con=engine).head()

,date,gdp_rate
0,1930,-8.5
1,1931,-6.4
2,1932,-12.9
3,1933,-1.2
4,1934,10.8


In [26]:
pd.read_sql_query('select * from m3', con=engine).head()

,date,m3_rate
0,1958,NaN
1,1959,NaN
2,1960,3.74
3,1961,6.75
4,1962,7.78
